# Brand Perception of British Airways & Emirates Input Processing Notebook  

## Abstract  
This notebook prepares the foundation for analyzing airline reviews of British Airways and Emirates.  
The workflow includes:  

- Extracting, cleaning, and structuring route information.  
- Generating origin, stopover, and destination fields.  
- Enriching routes with geocoded coordinates via Google Maps API.  
- Computing great-circle distances using the Haversine formula.  
- Categorizing flights into short-, medium-, and long-haul.

1. **Data Import & Setup**  
2. **Route Parsing**  
3. **Geocoding with Google Maps API**  
4. **Missing Data Checks**  
5. **Distance Calculation**  
6. **Flight Categorization**  
7. **Data Quality Summary**

# Load Datasets

In [1]:
import pandas as pd
# Load Emirates dataset
emirates_df = pd.read_csv(r"C:\Emirates Airways Reviews.csv")
# Load British Airways dataset
ba_df = pd.read_csv(r"C:\BA_AirlineReviews.csv")

In [2]:
# Print all column names in each dataset
print("British Airways Columns:")
for col in ba_df.columns:
    print("-", col)
print("\nEmirates Airways Columns:")
for col in emirates_df.columns:
    print("-", col)

British Airways Columns:
- Unnamed: 0
- OverallRating
- ReviewHeader
- Name
- Datetime
- VerifiedReview
- ReviewBody
- TypeOfTraveller
- SeatType
- Route
- DateFlown
- SeatComfort
- CabinStaffService
- GroundService
- ValueForMoney
- Recommended
- Aircraft
- Food&Beverages
- InflightEntertainment
- Wifi&Connectivity

Emirates Airways Columns:
- Title
- Date Published
- Status
- Aircraft
- Travel Type
- Travel Class
- Route
- Date Flown
- Seating Comfort
- Staff Service
- Food Quality
- Entertainment
- WiFi
- Ground Service
- Value for Money
- Recommended
- Overall Rating
- Review
- Country


- # Renaming Features

In [3]:
ba_rename_map = {
    "OverallRating": "overall_rating",
    "Datetime": "review_date",
    "ReviewBody": "review_text",
    "TypeOfTraveller": "travel_type",
    "SeatType": "travel_class",
    "DateFlown": "travel_date",
    "SeatComfort": "seat_comfort",
    "CabinStaffService": "staff_service",
    "GroundService": "ground_service",
    "ValueForMoney": "value_for_money",
    "Recommended": "recommended",
    "Food&Beverages": "food_quality",
    "InflightEntertainment": "entertainment",
    "Wifi&Connectivity": "wifi",
    "Aircraft": "aircraft",
    "Route": "route",
    "VerifiedReview": "verified_flag",
    "ReviewHeader": "review_title"
}
emirates_rename_map = {
    "Overall Rating": "overall_rating",
    "Date Published": "review_date",
    "Review": "review_text",
    "Travel Type": "travel_type",
    "Travel Class": "travel_class",
    "Date Flown": "travel_date",
    "Seating Comfort": "seat_comfort",
    "Staff Service": "staff_service",
    "Ground Service": "ground_service",
    "Value for Money": "value_for_money",
    "Recommended": "recommended",
    "Food Quality": "food_quality",
    "Entertainment": "entertainment",
    "WiFi": "wifi",
    "Aircraft": "aircraft",
    "Route": "route",
    "Status": "verified_flag",
    "Title": "review_title"
}


In [4]:
# Apply renaming
ba_df.rename(columns=ba_rename_map, inplace=True)
emirates_df.rename(columns=emirates_rename_map, inplace=True)

# Add airline source label for merging later
ba_df["airline"] = "British Airways"
emirates_df["airline"] = "Emirates"

# Preview new columns
print(" BA Columns After Renaming:\n", ba_df.columns.tolist())
print("Emirates Columns After Renaming:\n", emirates_df.columns.tolist())


 BA Columns After Renaming:
 ['Unnamed: 0', 'overall_rating', 'review_title', 'Name', 'review_date', 'verified_flag', 'review_text', 'travel_type', 'travel_class', 'route', 'travel_date', 'seat_comfort', 'staff_service', 'ground_service', 'value_for_money', 'recommended', 'aircraft', 'food_quality', 'entertainment', 'wifi', 'airline']
Emirates Columns After Renaming:
 ['review_title', 'review_date', 'verified_flag', 'aircraft', 'travel_type', 'travel_class', 'route', 'travel_date', 'seat_comfort', 'staff_service', 'food_quality', 'entertainment', 'wifi', 'ground_service', 'value_for_money', 'recommended', 'overall_rating', 'review_text', 'Country', 'airline']


- # Dropping unecessary columns

In [5]:
# Drop unnecessary columns from British Airways
ba_df.drop(columns=["Unnamed: 0", "Name"], inplace=True, errors='ignore')

# Drop unnecessary column from Emirates Airways
emirates_df.drop(columns=["Country"], inplace=True, errors='ignore')

# Confirm remaining columns
print("Remaining BA Columns:", ba_df.columns.tolist())
print("Remaining Emirates Columns:", emirates_df.columns.tolist())

Remaining BA Columns: ['overall_rating', 'review_title', 'review_date', 'verified_flag', 'review_text', 'travel_type', 'travel_class', 'route', 'travel_date', 'seat_comfort', 'staff_service', 'ground_service', 'value_for_money', 'recommended', 'aircraft', 'food_quality', 'entertainment', 'wifi', 'airline']
Remaining Emirates Columns: ['review_title', 'review_date', 'verified_flag', 'aircraft', 'travel_type', 'travel_class', 'route', 'travel_date', 'seat_comfort', 'staff_service', 'food_quality', 'entertainment', 'wifi', 'ground_service', 'value_for_money', 'recommended', 'overall_rating', 'review_text', 'airline']


In [6]:
ba_rename_map = {
    "OverallRating": "overall_rating",
    "Datetime": "review_date",
    "ReviewBody": "review_text",
    "TypeOfTraveller": "travel_type",
    "SeatType": "travel_class",
    "DateFlown": "travel_date",
    "SeatComfort": "seat_comfort",
    "CabinStaffService": "staff_service",
    "GroundService": "ground_service",
    "ValueForMoney": "value_for_money",
    "Recommended": "recommended",
    "Food&Beverages": "food_quality",
    "InflightEntertainment": "entertainment",
    "Wifi&Connectivity": "wifi",
    "Aircraft": "aircraft",
    "Route": "route",
    "VerifiedReview": "verified_flag",
    "ReviewHeader": "review_title"
}
emirates_rename_map = {
    "Overall Rating": "overall_rating",
    "Date Published": "review_date",
    "Review": "review_text",
    "Travel Type": "travel_type",
    "Travel Class": "travel_class",
    "Date Flown": "travel_date",
    "Seating Comfort": "seat_comfort",
    "Staff Service": "staff_service",
    "Ground Service": "ground_service",
    "Value for Money": "value_for_money",
    "Recommended": "recommended",
    "Food Quality": "food_quality",
    "Entertainment": "entertainment",
    "WiFi": "wifi",
    "Aircraft": "aircraft",
    "Route": "route",
    "Status": "verified_flag",
    "Title": "review_title"
}
# Apply renaming
ba_df.rename(columns=ba_rename_map, inplace=True)
emirates_df.rename(columns=emirates_rename_map, inplace=True)

# Add airline source label for merging later
ba_df["airline"] = "British Airways"
emirates_df["airline"] = "Emirates"

# Preview new columns
print(" BA Columns After Renaming:\n", ba_df.columns.tolist())
print("Emirates Columns After Renaming:\n", emirates_df.columns.tolist())


 BA Columns After Renaming:
 ['overall_rating', 'review_title', 'review_date', 'verified_flag', 'review_text', 'travel_type', 'travel_class', 'route', 'travel_date', 'seat_comfort', 'staff_service', 'ground_service', 'value_for_money', 'recommended', 'aircraft', 'food_quality', 'entertainment', 'wifi', 'airline']
Emirates Columns After Renaming:
 ['review_title', 'review_date', 'verified_flag', 'aircraft', 'travel_type', 'travel_class', 'route', 'travel_date', 'seat_comfort', 'staff_service', 'food_quality', 'entertainment', 'wifi', 'ground_service', 'value_for_money', 'recommended', 'overall_rating', 'review_text', 'airline']


# Review Date type Changed

In [7]:
# Check data type of 'review_date'
print("British Airways - review_date dtype:", ba_df['review_date'].dtype)
print("Emirates Airways - review_date dtype:", emirates_df['review_date'].dtype)

# Display first 5 review_date values
print("\nBritish Airways - First 5 review_date values:\n", ba_df['review_date'].head())
print("\nEmirates Airways - First 5 review_date values:\n", emirates_df['review_date'].head())

British Airways - review_date dtype: object
Emirates Airways - review_date dtype: object

British Airways - First 5 review_date values:
 0    19th November 2023
1    19th November 2023
2    16th November 2023
3    16th November 2023
4    14th November 2023
Name: review_date, dtype: object

Emirates Airways - First 5 review_date values:
 0    12th August 2024
1    11th August 2024
2      29th July 2024
3      18th July 2024
4      13th July 2024
Name: review_date, dtype: object


In [8]:
import re
from datetime import datetime

# Function to remove ordinal suffixes (st, nd, rd, th)
def clean_date_string(date_str):
    if pd.isnull(date_str):
        return None
    return re.sub(r'(\d{1,2})(st|nd|rd|th)', r'\1', date_str)

# Clean and convert for both datasets
ba_df['review_date'] = ba_df['review_date'].apply(clean_date_string)
emirates_df['review_date'] = emirates_df['review_date'].apply(clean_date_string)

# Convert to datetime
ba_df['review_date'] = pd.to_datetime(ba_df['review_date'], errors='coerce')
emirates_df['review_date'] = pd.to_datetime(emirates_df['review_date'], errors='coerce')

# Check types and preview
print("BA dtype:", ba_df['review_date'].dtype)
print("Emirates dtype:", emirates_df['review_date'].dtype)

print("\nBA review_date preview:\n", ba_df['review_date'].head())
print("\nEmirates review_date preview:\n", emirates_df['review_date'].head())

BA dtype: datetime64[ns]
Emirates dtype: datetime64[ns]

BA review_date preview:
 0   2023-11-19
1   2023-11-19
2   2023-11-16
3   2023-11-16
4   2023-11-14
Name: review_date, dtype: datetime64[ns]

Emirates review_date preview:
 0   2024-08-12
1   2024-08-11
2   2024-07-29
3   2024-07-18
4   2024-07-13
Name: review_date, dtype: datetime64[ns]


# Filtered Date betwee 2015 to 2024

In [9]:
# Filter reviews from 2015 to 2024 for British Airways
ba_df = ba_df[(ba_df['review_date'] >= '2015-01-01') & (ba_df['review_date'] <= '2024-12-31')]

# Filter reviews from 2015 to 2024 for Emirates Airways
emirates_df = emirates_df[(emirates_df['review_date'] >= '2015-01-01') & (emirates_df['review_date'] < '2024-1-1')]

# Confirm new min and max dates
print("British Airways - New Review Date Range:")
print(ba_df['review_date'].min(), "to", ba_df['review_date'].max())

print("\nEmirates Airways - New Review Date Range:")
print(emirates_df['review_date'].min(), "to", emirates_df['review_date'].max())

British Airways - New Review Date Range:
2015-01-05 00:00:00 to 2023-11-19 00:00:00

Emirates Airways - New Review Date Range:
2015-12-22 00:00:00 to 2023-12-31 00:00:00


In [111]:
# Filter reviews from 2015 to 2024 for British Airways
ba_df = ba_df[(ba_df['review_date'] >= '2015-01-01') & (ba_df['review_date'] <= '2024-12-31')]

# Filter reviews from 2015 to 2024 for Emirates Airways
emirates_df = emirates_df[(emirates_df['review_date'] >= '2015-01-01') & (emirates_df['review_date'] < '2024-1-1')]

# Confirm new min and max dates
print("British Airways - New Review Date Range:")
print(ba_df['review_date'].min(), "to", ba_df['review_date'].max())

print("\nEmirates Airways - New Review Date Range:")
print(emirates_df['review_date'].min(), "to", emirates_df['review_date'].max())

British Airways - New Review Date Range:
2015-01-05 00:00:00 to 2023-11-19 00:00:00

Emirates Airways - New Review Date Range:
2015-12-22 00:00:00 to 2023-12-31 00:00:00


# Conversion of Route to Filght Category [Short,Medium,Long,Unkow]

Extracted structured **origin** and **destination** fields from raw route strings by splitting on "to" and removing stopovers after "via".  

In [112]:
import re

def extract_origin_destination(route):
    # Lowercase for easier matching, then split logic
    if pd.isna(route):
        return None, None

    # Remove extra spaces
    route = route.strip()

    # Handle 'via' by splitting and taking the first part before 'via'
    if ' via ' in route.lower():
        main_part = route.lower().split(' via ')[0]
    else:
        main_part = route.lower()

    # Split by ' to '
    parts = re.split(r'\s+to\s+', main_part)

    # Origin is first element
    origin = parts[0].strip().title() if parts else None

    # Destination is last element (in case multiple 'to')
    destination = parts[-1].strip().title() if len(parts) > 1 else None

    return origin, destination

# Apply to your dataset
ba_df[['origin', 'destination']] = ba_df['route'].apply(
    lambda x: pd.Series(extract_origin_destination(x))
)

# Check result
print(ba_df[['route', 'origin', 'destination']].head(20))

                                  route           origin      destination
0                   London to Stuttgart           London        Stuttgart
1                    Brussels to London         Brussels           London
2             London Heathrow to Dublin  London Heathrow           Dublin
3                      London to Dublin           London           Dublin
4                      London to Lisbon           London           Lisbon
5    Bucharest to Manchester via London        Bucharest       Manchester
6    Manchester to Cape Town via London       Manchester        Cape Town
7            Seville to London Gatwick           Seville   London Gatwick
8             London Heatrow to Atlanta   London Heatrow          Atlanta
9                    Gatwick to Antalya          Gatwick          Antalya
10            Dublin to London Heathrow           Dublin  London Heathrow
11  Anchorage to Heathrow via Barcelona        Anchorage         Heathrow
12         London Heathrow to Gibralta

In [113]:
# Example: uppercase and strip spaces
ba_df['origin'] = ba_df['origin'].str.upper().str.strip()
ba_df['destination'] = ba_df['destination'].str.upper().str.strip()

In [114]:
import re
import pandas as pd

# Function to extract origin and destination from route strings
def extract_origin_destination(route):
    if pd.isna(route):
        return None, None

    route = route.strip()

    # Handle 'via' by splitting and taking only the main part before 'via'
    if ' via ' in route.lower():
        main_part = route.lower().split(' via ')[0]
    else:
        main_part = route.lower()

    # Split by ' to '
    parts = re.split(r'\s+to\s+', main_part)

    origin = parts[0].strip().title() if parts else None
    destination = parts[-1].strip().title() if len(parts) > 1 else None

    return origin, destination

# Apply to Emirates dataset
emirates_df[['origin', 'destination']] = emirates_df['route'].apply(
    lambda x: pd.Series(extract_origin_destination(x))
)

# Preview result
print(emirates_df[['route', 'origin', 'destination']].head(20))

                                      route           origin       destination
65            Heathrow to Bangkok via Dubai         Heathrow           Bangkok
66            Boston to Bangalore via Dubai           Boston         Bangalore
67       Montreal to Kuala Lumpur via Dubai         Montreal      Kuala Lumpur
68              Glasgow to Narita via Dubai          Glasgow            Narita
69                            Oslo to Dubai             Oslo             Dubai
70        Dubai to Queenstown via Melbourne            Dubai        Queenstown
71           Bahrain to Mogadishu via Dubai          Bahrain         Mogadishu
72              Glasgow to Narita via Dubai          Glasgow            Narita
73                         Toronto to Dubai          Toronto             Dubai
74                       Barcelona to Dubai        Barcelona             Dubai
75           Frankfurt to Colombo via Dubai        Frankfurt           Colombo
76                    Sydney to Birmingham          

# Route Parsing, Geocoding & Flight Categorization
- Parsed raw route strings into `origin`, `stopover`, and `destination`.  
- Used Google Maps Geocoding API to fetch latitude/longitude coordinates for each airport.  
- Checked for missing coordinates and summarized missingness.  
- Calculated great-circle flight distance (km) between origin and destination using the Haversine formula.  
- Classified flights into categories:
  - **Short-haul:** < 1500 km  
  - **Medium-haul:** 1500–3500 km  
  - **Long-haul:** > 3500 km  

 Outcome: Each review now has structured origin/stopover/destination, geocoded coordinates, computed flight distance, and a flight category label.


In [117]:
pip install googlemaps

Note: you may need to restart the kernel to use updated packages.


- # Emirates Route Parsing

In [118]:
import re
import pandas as pd
import googlemaps

# Google API key
API_KEY = "AIzaSyBYL1ms******************************"
gmaps = googlemaps.Client(key=API_KEY)

# Extract origin / stopover / destination from route
def extract_route_parts(route):
    if pd.isna(route):
        return None, None, None
    stopover = None
    if ' via ' in route.lower():
        main_part, stopover_part = route.split(' via ', 1)
        stopover = stopover_part.strip()
    else:
        main_part = route
    parts = re.split(r'\s+to\s+', main_part, flags=re.IGNORECASE)
    origin = parts[0].strip() if parts else None
    destination = parts[-1].strip() if len(parts) > 1 else None
    return origin, stopover, destination

# Get coordinates from Google Geocoding API
def get_coordinates(place):
    if not place:
        return None
    try:
        query = f"{place} airport"
        result = gmaps.geocode(query)
        if result and 'geometry' in result[0]:
            loc = result[0]['geometry']['location']
            return (loc['lat'], loc['lng'])
    except Exception as e:
        print(f"Error geocoding {place}: {e}")
    return None

# Apply to Emirates dataset
emirates_df[['origin_raw', 'stopover_raw', 'destination_raw']] = emirates_df['route'].apply(
    lambda x: pd.Series(extract_route_parts(x))
)

# Get coordinates directly from Google
emirates_df['origin_coords'] = emirates_df['origin_raw'].apply(get_coordinates)
emirates_df['stopover_coords'] = emirates_df['stopover_raw'].apply(get_coordinates)
emirates_df['destination_coords'] = emirates_df['destination_raw'].apply(get_coordinates)

# Preview
print(emirates_df[['route', 'origin_raw', 'origin_coords', 'stopover_raw', 'stopover_coords', 'destination_raw', 'destination_coords']].head())

                                 route origin_raw  \
65       Heathrow to Bangkok via Dubai   Heathrow   
66       Boston to Bangalore via Dubai     Boston   
67  Montreal to Kuala Lumpur via Dubai   Montreal   
68         Glasgow to Narita via Dubai    Glasgow   
69                       Oslo to Dubai       Oslo   

                      origin_coords stopover_raw  \
65          (51.4679914, -0.455051)        Dubai   
66         (42.365602, -71.0096136)        Dubai   
67        (45.4578591, -73.7493009)        Dubai   
68  (55.87004150000001, -4.4345433)        Dubai   
69         (60.1978446, 11.0963701)         None   

                    stopover_coords destination_raw  \
65  (25.2566932, 55.36431779999999)         Bangkok   
66  (25.2566932, 55.36431779999999)       Bangalore   
67  (25.2566932, 55.36431779999999)    Kuala Lumpur   
68  (25.2566932, 55.36431779999999)          Narita   
69                             None           Dubai   

                 destination_coords  

In [119]:
# Find rows where any coordinate is missing
missing_coords = emirates_df[
    emirates_df['origin_coords'].isna() |
    emirates_df['stopover_coords'].isna() |
    emirates_df['destination_coords'].isna()
]

# See how many are missing
print(f"Total routes with missing coordinates: {len(missing_coords)}")

# See the actual missing rows
print(missing_coords[['route', 'origin_raw', 'origin_coords', 'stopover_raw', 'stopover_coords', 'destination_raw', 'destination_coords']])

Total routes with missing coordinates: 626
                       route    origin_raw                     origin_coords  \
69             Oslo to Dubai          Oslo          (60.1978446, 11.0963701)   
73          Toronto to Dubai       Toronto  (43.6798345, -79.62838339999999)   
74        Barcelona to Dubai     Barcelona    (41.29834049999999, 2.0800095)   
76     Sydney to Birmingham         Sydney         (-33.950033, 151.1816955)   
77    Christchurch to Sydney  Christchurch          (-43.48763, 172.5374026)   
...                      ...           ...                               ...   
1533              DXB to NBO           DXB   (25.2566932, 55.36431779999999)   
1534              DXB to BKK           DXB   (25.2566932, 55.36431779999999)   
1536              BHX to DXB           BHX          (52.4523739, -1.7435082)   
1538              DXB to JED           DXB   (25.2566932, 55.36431779999999)   
1539              BKK to HKG           BKK         (13.6818969, 100.7468694) 

In [120]:
# Count missing per column
missing_summary = {
    "Origin missing": emirates_df['origin_coords'].isna().sum(),
    "Stopover missing": emirates_df['stopover_coords'].isna().sum(),
    "Destination missing": emirates_df['destination_coords'].isna().sum()
}

print(missing_summary)

{'Origin missing': 2, 'Stopover missing': 625, 'Destination missing': 3}


In [121]:
# Show routes with missing origin or destination coordinates
problem_routes = emirates_df[
    emirates_df['origin_coords'].isna() | emirates_df['destination_coords'].isna()
]

print(problem_routes[['route', 'origin_raw', 'origin_coords', 'destination_raw', 'destination_coords']])

                                     route origin_raw  \
99                                     NaN       None   
371                                    NaN       None   
1157  Hong Kong to Arla via Arlanda Sweden  Hong Kong   

                  origin_coords destination_raw destination_coords  
99                         None            None               None  
371                        None            None               None  
1157  (22.3134736, 113.9137283)            Arla               None  


In [122]:
# Fix "Arla" -> "Arlanda Airport Sweden"
emirates_df.at[1157, 'destination_raw'] = "Arlanda Airport Sweden"
# Assign coordinates as a single tuple object
emirates_df.at[1157, 'destination_coords'] = tuple(get_coordinates("Arlanda Airport Sweden"))
emirates_df['route'] = emirates_df['route'].fillna('unknown')

In [123]:
# Count missing per column
missing_summary = {
    "Origin missing": emirates_df['origin_coords'].isna().sum(),
    "Stopover missing": emirates_df['stopover_coords'].isna().sum(),
    "Destination missing": emirates_df['destination_coords'].isna().sum()
}

print(missing_summary)

{'Origin missing': 2, 'Stopover missing': 625, 'Destination missing': 2}


In [124]:
import math

# Haversine formula to calculate great-circle distance (in km)
def haversine(coord1, coord2):
    if not coord1 or not coord2:
        return None
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    R = 6371  # Earth radius in km
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda/2)**2
    return R * (2 * math.atan2(math.sqrt(a), math.sqrt(1-a)))

# Apply distance calculation
emirates_df['distance_km'] = emirates_df.apply(
    lambda row: haversine(row['origin_coords'], row['destination_coords']),
    axis=1
)

# Classification rules
def classify_flight(distance):
    if distance is None or pd.isna(distance):
        return "Unknown"
    elif distance < 1500:
        return "Short-haul"
    elif distance < 3500:
        return "Medium-haul"
    else:
        return "Long-haul"

# Apply classification
emirates_df['flight_category'] = emirates_df['distance_km'].apply(classify_flight)

# Preview
print(emirates_df[['route', 'origin_coords', 'destination_coords', 'distance_km', 'flight_category']].head(20))

                                      route                     origin_coords  \
65            Heathrow to Bangkok via Dubai           (51.4679914, -0.455051)   
66            Boston to Bangalore via Dubai          (42.365602, -71.0096136)   
67       Montreal to Kuala Lumpur via Dubai         (45.4578591, -73.7493009)   
68              Glasgow to Narita via Dubai   (55.87004150000001, -4.4345433)   
69                            Oslo to Dubai          (60.1978446, 11.0963701)   
70        Dubai to Queenstown via Melbourne   (25.2566932, 55.36431779999999)   
71           Bahrain to Mogadishu via Dubai          (26.2671847, 50.6302827)   
72              Glasgow to Narita via Dubai   (55.87004150000001, -4.4345433)   
73                         Toronto to Dubai  (43.6798345, -79.62838339999999)   
74                       Barcelona to Dubai    (41.29834049999999, 2.0800095)   
75           Frankfurt to Colombo via Dubai           (50.1109221, 8.6821267)   
76                    Sydney

In [125]:
# Count of each flight category
category_counts = emirates_df['flight_category'].value_counts()
print(category_counts)

flight_category
Long-haul      1314
Medium-haul     139
Short-haul       20
Unknown           2
Name: count, dtype: int64


- # British Airways Route Parsing

In [126]:
import re
import pandas as pd
import googlemaps

# Google API key
API_KEY = "AIzaSyBYL1ms************************"
gmaps = googlemaps.Client(key=API_KEY)

# Extract origin / stopover / destination from route
def extract_route_parts(route):
    if pd.isna(route) or route.strip().lower() == "unknown":
        return None, None, None
    stopover = None
    if ' via ' in route.lower():
        main_part, stopover_part = route.split(' via ', 1)
        stopover = stopover_part.strip()
    else:
        main_part = route
    parts = re.split(r'\s+to\s+', main_part, flags=re.IGNORECASE)
    origin = parts[0].strip() if parts else None
    destination = parts[-1].strip() if len(parts) > 1 else None
    return origin, stopover, destination

# Cache to avoid repeated API calls
coord_cache = {}

# Get coordinates from Google Geocoding API
def get_coordinates(place):
    if not place or place.strip().lower() == "unknown":
        return None
    if place in coord_cache:
        return coord_cache[place]
    try:
        query = f"{place} airport"
        result = gmaps.geocode(query)
        if result and 'geometry' in result[0]:
            loc = result[0]['geometry']['location']
            coords = (loc['lat'], loc['lng'])
            coord_cache[place] = coords
            return coords
    except Exception as e:
        print(f"Error geocoding {place}: {e}")
    coord_cache[place] = None
    return None

# Apply to BA dataset
ba_df[['origin_raw', 'stopover_raw', 'destination_raw']] = ba_df['route'].apply(
    lambda x: pd.Series(extract_route_parts(x))
)

# Get coordinates
ba_df['origin_coords'] = ba_df['origin_raw'].apply(get_coordinates)
ba_df['stopover_coords'] = ba_df['stopover_raw'].apply(get_coordinates)
ba_df['destination_coords'] = ba_df['destination_raw'].apply(get_coordinates)

# Preview
print(ba_df[['route', 'origin_raw', 'origin_coords', 'stopover_raw', 'stopover_coords', 'destination_raw', 'destination_coords']].head())

                       route       origin_raw            origin_coords  \
0        London to Stuttgart           London  (51.4679914, -0.455051)   
1         Brussels to London         Brussels    (50.90024, 4.4859439)   
2  London Heathrow to Dublin  London Heathrow  (51.4679914, -0.455051)   
3           London to Dublin           London  (51.4679914, -0.455051)   
4           London to Lisbon           London  (51.4679914, -0.455051)   

  stopover_raw stopover_coords destination_raw  \
0         None            None       Stuttgart   
1         None            None          London   
2         None            None          Dublin   
3         None            None          Dublin   
4         None            None          Lisbon   

                destination_coords  
0          (48.6858082, 9.2073536)  
1          (51.4679914, -0.455051)  
2  (53.425632, -6.257375499999999)  
3  (53.425632, -6.257375499999999)  
4         (38.7788454, -9.1319758)  


In [127]:
# Find rows where any coordinate is missing
missing_coords_ba = ba_df[
    ba_df['origin_coords'].isna() |
    ba_df['stopover_coords'].isna() |
    ba_df['destination_coords'].isna()
]

# See how many are missing
print(f"Total routes with missing coordinates in BA: {len(missing_coords_ba)}")

# See the actual missing rows
print(missing_coords_ba[['route', 'origin_raw', 'origin_coords', 
                         'stopover_raw', 'stopover_coords', 
                         'destination_raw', 'destination_coords']])

Total routes with missing coordinates in BA: 2717
                          route       origin_raw            origin_coords  \
0           London to Stuttgart           London  (51.4679914, -0.455051)   
1            Brussels to London         Brussels    (50.90024, 4.4859439)   
2     London Heathrow to Dublin  London Heathrow  (51.4679914, -0.455051)   
3              London to Dublin           London  (51.4679914, -0.455051)   
4              London to Lisbon           London  (51.4679914, -0.455051)   
...                         ...              ...                      ...   
3213                        NaN             None                     None   
3214                        NaN             None                     None   
3215                        NaN             None                     None   
3216                        NaN             None                     None   
3217                        NaN             None                     None   

     stopover_raw stopove

In [128]:
# Count missing per column for BA
missing_summary_ba = {
    "Origin missing": ba_df['origin_coords'].isna().sum(),
    "Stopover missing": ba_df['stopover_coords'].isna().sum(),
    "Destination missing": ba_df['destination_coords'].isna().sum()
}

print(missing_summary_ba)

{'Origin missing': 293, 'Stopover missing': 2711, 'Destination missing': 312}


In [129]:
# Show BA routes with missing origin or destination coordinates
problem_routes_ba = ba_df[
    ba_df['origin_coords'].isna() | ba_df['destination_coords'].isna()
]

print(problem_routes_ba[['route', 'origin_raw', 'origin_coords', 
                         'destination_raw', 'destination_coords']])

                                         route        origin_raw  \
22                         Istanbul via London          Istanbul   
135   Sydney via Singapore to London Heathrow             Sydney   
159                                        NaN              None   
220                          London Singapore   London Singapore   
351                                        NaN              None   
...                                        ...               ...   
3213                                       NaN              None   
3214                                       NaN              None   
3215                                       NaN              None   
3216                                       NaN              None   
3217                                       NaN              None   

                  origin_coords destination_raw destination_coords  
22     (41.2768187, 28.7301397)            None               None  
135   (-33.950033, 151.1816955)            No

In [130]:
# Filter BA routes where origin or destination coords are missing but route is NOT 'Unknown'
problem_routes_ba_non_unknown = ba_df[
    ((ba_df['origin_coords'].isna()) | (ba_df['destination_coords'].isna())) &
    (ba_df['route'] != "Unknown")
]

print(f"Number of non-'Unknown' BA routes missing coords: {len(problem_routes_ba_non_unknown)}")

# Show them
print(problem_routes_ba_non_unknown[['route', 'origin_raw', 'origin_coords', 
                                     'destination_raw', 'destination_coords']])

Number of non-'Unknown' BA routes missing coords: 313
                                         route        origin_raw  \
22                         Istanbul via London          Istanbul   
135   Sydney via Singapore to London Heathrow             Sydney   
159                                        NaN              None   
220                          London Singapore   London Singapore   
351                                        NaN              None   
...                                        ...               ...   
3213                                       NaN              None   
3214                                       NaN              None   
3215                                       NaN              None   
3216                                       NaN              None   
3217                                       NaN              None   

                  origin_coords destination_raw destination_coords  
22     (41.2768187, 28.7301397)            None             

In [131]:
# Manual destination fixes
fixes = {
    22: "London Heathrow Airport",
    135: "London Heathrow Airport",
    220: "Singapore Changi Airport",
    1148: "Florence Airport Italy",
    2344: "Incheon International Airport",
    2476: "Lisbon Airport Portugal",
    2836: "London Heathrow Airport",
    2848: "Mumbai Airport India",
    2885: "Taveuni Airport Fiji", 
    2920: "Denver International Airport",
    2921: "London Heathrow Airport",
    2922: "Glasgow Airport Scotland",
    2923: "London Heathrow Airport",
    2924: "London Heathrow Airport",
    2928: "Vancouver International Airport",
    2929: "Chicago O'Hare International Airport",
    2930: "London Heathrow Airport",
    2931: "Mumbai Airport India"
}

# Apply fixes & get coordinates
for idx, dest in fixes.items():
    ba_df.at[idx, 'destination_raw'] = dest
    ba_df.at[idx, 'destination_coords'] = get_coordinates(dest)

# Preview to check
print(ba_df.loc[fixes.keys(), ['route', 'destination_raw', 'destination_coords']])

                                         route  \
22                         Istanbul via London   
135   Sydney via Singapore to London Heathrow    
220                          London Singapore    
1148                       Edinburgh-Florence    
2344                        BCN to SEL via LHR   
2476                           YVR-LIS via LHR   
2836                                  MAN-LHR    
2848                                   LHR-BOM   
2885                        JFK to TAV via LHR   
2920                                   LHR-DEN   
2921                                   ORD-LHR   
2922                                   LHR-GLA   
2923                                   GLA-LHR   
2924                                  FRA-LHR    
2928                   London-Vancouver return   
2929                                   LHR-ORD   
2930                                   BOM-LHR   
2931                                   LHR-BOM   

                           destination_raw       

In [132]:
# Count missing per column for BA
missing_summary_ba = {
    "Origin missing": ba_df['origin_coords'].isna().sum(),
    "Stopover missing": ba_df['stopover_coords'].isna().sum(),
    "Destination missing": ba_df['destination_coords'].isna().sum()
}

print(missing_summary_ba)

{'Origin missing': 293, 'Stopover missing': 2711, 'Destination missing': 294}


In [133]:
# Fix missing origin for Port of Spain
ba_df.at[1457, 'origin_raw'] = "Piarco International Airport Trinidad"
ba_df.at[1457, 'origin_coords'] = get_coordinates("Piarco International Airport Trinidad")

In [134]:
# Count missing per column for BA
missing_summary_ba = {
    "Origin missing": ba_df['origin_coords'].isna().sum(),
    "Stopover missing": ba_df['stopover_coords'].isna().sum(),
    "Destination missing": ba_df['destination_coords'].isna().sum()
}

print(missing_summary_ba)

{'Origin missing': 293, 'Stopover missing': 2711, 'Destination missing': 294}


In [135]:
import math
import pandas as pd

# --- 1) Normalize route column ---
ba_df['route'] = ba_df['route'].fillna("Unknown").str.strip().str.title()

# --- 2) Haversine formula ---
def haversine(coord1, coord2):
    if not coord1 or not coord2 or pd.isna(coord1) or pd.isna(coord2):
        return None
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    R = 6371  # Earth radius in km
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda/2)**2
    return R * (2 * math.atan2(math.sqrt(a), math.sqrt(1-a)))

# --- 3) Calculate distance, skipping unknowns ---
ba_df['distance_km'] = ba_df.apply(
    lambda row: None if row['route'] == "Unknown" 
    or pd.isna(row['origin_coords']) 
    or pd.isna(row['destination_coords'])
    else haversine(row['origin_coords'], row['destination_coords']),
    axis=1
)

# --- 4) Classification function ---
def classify_flight(distance, route):
    if route == "Unknown" or pd.isna(distance):
        return "Unknown"
    elif distance < 1500:
        return "Short-haul"
    elif distance < 3500:
        return "Medium-haul"
    else:
        return "Long-haul"

# --- 5) Apply classification ---
ba_df['flight_category'] = ba_df.apply(
    lambda row: classify_flight(row['distance_km'], row['route']),
    axis=1
)

# --- 6) Preview result ---
print(ba_df[['route', 'origin_coords', 'destination_coords', 'distance_km', 'flight_category']].head(20))

                                  route                      origin_coords  \
0                   London To Stuttgart            (51.4679914, -0.455051)   
1                    Brussels To London              (50.90024, 4.4859439)   
2             London Heathrow To Dublin            (51.4679914, -0.455051)   
3                      London To Dublin            (51.4679914, -0.455051)   
4                      London To Lisbon            (51.4679914, -0.455051)   
5    Bucharest To Manchester Via London           (44.4267674, 26.1025384)   
6    Manchester To Cape Town Via London            (53.3553569, -2.277162)   
7             Seville To London Gatwick            (37.4202698, -5.890794)   
8             London Heatrow To Atlanta            (51.4679914, -0.455051)   
9                    Gatwick To Antalya           (51.1536621, -0.1820629)   
10            Dublin To London Heathrow    (53.425632, -6.257375499999999)   
11  Anchorage To Heathrow Via Barcelona         (61.2175758, -14

In [136]:
# Count of each flight category
category_count_ba = ba_df['flight_category'].value_counts()

print(category_count_ba)

flight_category
Long-haul      1765
Short-haul      761
Medium-haul     397
Unknown         295
Name: count, dtype: int64


In [139]:
# Save full updated British Airways dataset
ba_output_path = r"C:\Users\Samiksha\Downloads\BA_updated.xlsx"
ba_df.to_excel(ba_output_path, index=False)
print(f"British Airways data saved to {ba_output_path}")

# Save full updated Emirates dataset
emirates_output_path = r"C:\Users\Samiksha\Downloads\Emirates_updated.xlsx"
emirates_df.to_excel(emirates_output_path, index=False)
print(f"Emirates Airways data saved to {emirates_output_path}")

British Airways data saved to C:\Users\Samiksha\Downloads\BA_updated.xlsx
Emirates Airways data saved to C:\Users\Samiksha\Downloads\Emirates_updated.xlsx


# Assigning Periods

In [155]:
import pandas as pd

def add_covid_period(df, date_col='ReviewDate'):
    # Ensure datetime
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
    
    # Define COVID cutoffs
    pre_covid_end = pd.Timestamp('2020-02-29')
    covid_end = pd.Timestamp('2022-02-28')
    
    # Assign periods
    df['covid_period'] = pd.cut(
        df[date_col],
        bins=[pd.Timestamp('1900-01-01'), pre_covid_end, covid_end, pd.Timestamp('2100-01-01')],
        labels=['Pre-COVID', 'During-COVID', 'Post-COVID'],
        right=True
    )
    return df

# Apply to both datasets
ba_df = add_covid_period(ba_df, 'review_date')

# Quick check
print(ba_df['covid_period'].value_counts())


covid_period
Pre-COVID       2608
Post-COVID       442
During-COVID     168
Name: count, dtype: int64
